In [6]:
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from utils import *
from descriptor import *
import warnings

warnings.filterwarnings("ignore")

train_file = "../features/train_data.csv"
test_file = "../features/test_data.csv"

train_data = pd.read_csv(train_file,index_col=0)
test_data = pd.read_csv(test_file,index_col=0)

data = pd.concat([train_data,test_data],axis=0)

property_ = ["GVRH"]
soap_columns = list(data.columns)[198:]

columns = geometric + RAC + soap_columns + property_
new_columns = geometric + RAC + soap_columns
descriptors = data[new_columns]

drop_col = remove_constant_value_features(descriptors)
new_df_columns = [e for e in descriptors.columns if e not in drop_col]

# X_train = np.array(train_data[new_df_columns])
# X_test = np.array(test_data[new_df_columns])
# y_train = np.array(train_data[property_])
# y_test = np.array(test_data[property_])

X_train = train_data[new_df_columns]
X_test = test_data[new_df_columns]
y_train = train_data[property_]
y_test = test_data[property_]

In [ ]:
import optuna
from sklearn import datasets

def objective(trial,X_train = X_train, y_train = y_train):
    
        
    learning_rate = trial.suggest_discrete_uniform("learning_rate", 0.001, 0.1, 0.005)
  
    depth = trial.suggest_int('depth', 5, 10)
                
    l2_leaf_reg = trial.suggest_discrete_uniform('l2_leaf_reg', 2.0, 30.0, 0.5)
        
    min_child_samples = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
        
    grow_policy = trial.suggest_categorical("grow_policy",["SymmetricTree", "Depthwise", "Lossguide"])
    
    accuracy = []
    
    kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

    loss = []

    for train_index2, valid_index2 in kf.split(X_train):

        X_train2, X_valid2 = X_train[train_index2], X_train[valid_index2]
        y_train2, y_valid2 = y_train[train_index2], y_train[valid_index2]

        model = CatBoostRegressor(random_state=42,
                                  learning_rate=learning_rate,
                                  depth = depth,
                                  l2_leaf_reg = l2_leaf_reg,
                                  min_child_samples = min_child_samples,
                                  grow_policy = grow_policy,
                                  silent = True) 

        model.fit(X_train2, y_train2) 

        y_pred2 = model.predict(X_valid2) 
        
        score = r2_score(y_valid2,y_pred2)
        
        rmse = mean_squared_error(y_valid2,y_pred2,squared = False)
        
        loss.append(rmse)
        
        accuracy.append(score)
        
    average_loss = np.mean(loss)
    average_score = np.mean(accuracy)    
    y_pred = model.predict(X_test)
    
    print(r2_score(y_test,y_pred))
    
    return average_score
    
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials = 1000)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print("best r2 is :",study.best_trial.values)


In [7]:
params = {'learning_rate': 0.056, 
          'depth': 10, 
          'l2_leaf_reg': 2.0, 
          'min_child_samples': 1,
          'grow_policy': 'Lossguide'}

In [8]:
model = CatBoostRegressor(random_state=42)
model.fit(X_train,y_train)

Learning rate set to 0.058198
0:	learn: 3.1551222	total: 438ms	remaining: 7m 17s
1:	learn: 3.0955916	total: 682ms	remaining: 5m 40s
2:	learn: 3.0377874	total: 908ms	remaining: 5m 1s
3:	learn: 2.9824195	total: 1.16s	remaining: 4m 47s
4:	learn: 2.9262680	total: 1.38s	remaining: 4m 34s
5:	learn: 2.8766432	total: 1.65s	remaining: 4m 33s
6:	learn: 2.8277184	total: 1.88s	remaining: 4m 26s
7:	learn: 2.7839535	total: 2.11s	remaining: 4m 21s
8:	learn: 2.7422528	total: 2.35s	remaining: 4m 18s
9:	learn: 2.7059767	total: 2.58s	remaining: 4m 15s
10:	learn: 2.6700140	total: 2.83s	remaining: 4m 14s
11:	learn: 2.6335341	total: 3.06s	remaining: 4m 12s
12:	learn: 2.6016314	total: 3.32s	remaining: 4m 12s
13:	learn: 2.5723539	total: 3.56s	remaining: 4m 11s
14:	learn: 2.5452982	total: 3.79s	remaining: 4m 9s
15:	learn: 2.5196921	total: 4.02s	remaining: 4m 7s
16:	learn: 2.4956824	total: 4.26s	remaining: 4m 6s
17:	learn: 2.4705986	total: 4.51s	remaining: 4m 6s
18:	learn: 2.4481340	total: 4.74s	remaining: 4m 4

157:	learn: 1.7250079	total: 36.8s	remaining: 3m 16s
158:	learn: 1.7226045	total: 37s	remaining: 3m 15s
159:	learn: 1.7190050	total: 37.2s	remaining: 3m 15s
160:	learn: 1.7168109	total: 37.5s	remaining: 3m 15s
161:	learn: 1.7118691	total: 37.7s	remaining: 3m 15s
162:	learn: 1.7104643	total: 37.9s	remaining: 3m 14s
163:	learn: 1.7086126	total: 38.2s	remaining: 3m 14s
164:	learn: 1.7059328	total: 38.4s	remaining: 3m 14s
165:	learn: 1.7006852	total: 38.6s	remaining: 3m 14s
166:	learn: 1.6981923	total: 38.8s	remaining: 3m 13s
167:	learn: 1.6953723	total: 39.1s	remaining: 3m 13s
168:	learn: 1.6915664	total: 39.3s	remaining: 3m 13s
169:	learn: 1.6881605	total: 39.5s	remaining: 3m 12s
170:	learn: 1.6871298	total: 39.7s	remaining: 3m 12s
171:	learn: 1.6846724	total: 40s	remaining: 3m 12s
172:	learn: 1.6837624	total: 40.2s	remaining: 3m 12s
173:	learn: 1.6824585	total: 40.4s	remaining: 3m 11s
174:	learn: 1.6790995	total: 40.6s	remaining: 3m 11s
175:	learn: 1.6757471	total: 40.8s	remaining: 3m 1

313:	learn: 1.3734826	total: 1m 12s	remaining: 2m 38s
314:	learn: 1.3729978	total: 1m 12s	remaining: 2m 38s
315:	learn: 1.3706060	total: 1m 13s	remaining: 2m 38s
316:	learn: 1.3701085	total: 1m 13s	remaining: 2m 38s
317:	learn: 1.3684220	total: 1m 13s	remaining: 2m 37s
318:	learn: 1.3680832	total: 1m 13s	remaining: 2m 37s
319:	learn: 1.3658483	total: 1m 14s	remaining: 2m 37s
320:	learn: 1.3635811	total: 1m 14s	remaining: 2m 37s
321:	learn: 1.3617942	total: 1m 14s	remaining: 2m 37s
322:	learn: 1.3597539	total: 1m 14s	remaining: 2m 36s
323:	learn: 1.3591585	total: 1m 15s	remaining: 2m 36s
324:	learn: 1.3572706	total: 1m 15s	remaining: 2m 36s
325:	learn: 1.3556277	total: 1m 15s	remaining: 2m 36s
326:	learn: 1.3530242	total: 1m 15s	remaining: 2m 35s
327:	learn: 1.3511303	total: 1m 15s	remaining: 2m 35s
328:	learn: 1.3504123	total: 1m 16s	remaining: 2m 35s
329:	learn: 1.3496758	total: 1m 16s	remaining: 2m 35s
330:	learn: 1.3478864	total: 1m 16s	remaining: 2m 34s
331:	learn: 1.3450790	total:

466:	learn: 1.1665812	total: 1m 48s	remaining: 2m 3s
467:	learn: 1.1653082	total: 1m 48s	remaining: 2m 3s
468:	learn: 1.1638019	total: 1m 48s	remaining: 2m 2s
469:	learn: 1.1621512	total: 1m 48s	remaining: 2m 2s
470:	learn: 1.1603417	total: 1m 48s	remaining: 2m 2s
471:	learn: 1.1600924	total: 1m 49s	remaining: 2m 2s
472:	learn: 1.1586770	total: 1m 49s	remaining: 2m 1s
473:	learn: 1.1584669	total: 1m 49s	remaining: 2m 1s
474:	learn: 1.1582261	total: 1m 49s	remaining: 2m 1s
475:	learn: 1.1579092	total: 1m 50s	remaining: 2m 1s
476:	learn: 1.1576777	total: 1m 50s	remaining: 2m
477:	learn: 1.1574517	total: 1m 50s	remaining: 2m
478:	learn: 1.1559657	total: 1m 50s	remaining: 2m
479:	learn: 1.1546556	total: 1m 50s	remaining: 2m
480:	learn: 1.1543538	total: 1m 51s	remaining: 1m 59s
481:	learn: 1.1529656	total: 1m 51s	remaining: 1m 59s
482:	learn: 1.1505955	total: 1m 51s	remaining: 1m 59s
483:	learn: 1.1492354	total: 1m 51s	remaining: 1m 59s
484:	learn: 1.1478281	total: 1m 52s	remaining: 1m 59s


620:	learn: 1.0057640	total: 2m 23s	remaining: 1m 27s
621:	learn: 1.0048045	total: 2m 23s	remaining: 1m 27s
622:	learn: 1.0038252	total: 2m 23s	remaining: 1m 26s
623:	learn: 1.0035202	total: 2m 23s	remaining: 1m 26s
624:	learn: 1.0027840	total: 2m 24s	remaining: 1m 26s
625:	learn: 1.0017462	total: 2m 24s	remaining: 1m 26s
626:	learn: 0.9997372	total: 2m 24s	remaining: 1m 26s
627:	learn: 0.9987035	total: 2m 24s	remaining: 1m 25s
628:	learn: 0.9984714	total: 2m 25s	remaining: 1m 25s
629:	learn: 0.9983136	total: 2m 25s	remaining: 1m 25s
630:	learn: 0.9971254	total: 2m 25s	remaining: 1m 25s
631:	learn: 0.9968368	total: 2m 25s	remaining: 1m 24s
632:	learn: 0.9964484	total: 2m 25s	remaining: 1m 24s
633:	learn: 0.9955602	total: 2m 26s	remaining: 1m 24s
634:	learn: 0.9942678	total: 2m 26s	remaining: 1m 24s
635:	learn: 0.9936379	total: 2m 26s	remaining: 1m 23s
636:	learn: 0.9929690	total: 2m 26s	remaining: 1m 23s
637:	learn: 0.9916379	total: 2m 27s	remaining: 1m 23s
638:	learn: 0.9903422	total:

774:	learn: 0.8838339	total: 2m 58s	remaining: 51.9s
775:	learn: 0.8836200	total: 2m 58s	remaining: 51.7s
776:	learn: 0.8830312	total: 2m 59s	remaining: 51.4s
777:	learn: 0.8823673	total: 2m 59s	remaining: 51.2s
778:	learn: 0.8810667	total: 2m 59s	remaining: 51s
779:	learn: 0.8800683	total: 2m 59s	remaining: 50.7s
780:	learn: 0.8793250	total: 3m	remaining: 50.5s
781:	learn: 0.8785672	total: 3m	remaining: 50.3s
782:	learn: 0.8780672	total: 3m	remaining: 50.1s
783:	learn: 0.8779610	total: 3m	remaining: 49.8s
784:	learn: 0.8769588	total: 3m 1s	remaining: 49.6s
785:	learn: 0.8766509	total: 3m 1s	remaining: 49.4s
786:	learn: 0.8757288	total: 3m 1s	remaining: 49.1s
787:	learn: 0.8755657	total: 3m 1s	remaining: 48.9s
788:	learn: 0.8746375	total: 3m 2s	remaining: 48.7s
789:	learn: 0.8744414	total: 3m 2s	remaining: 48.4s
790:	learn: 0.8736541	total: 3m 2s	remaining: 48.2s
791:	learn: 0.8726034	total: 3m 2s	remaining: 48s
792:	learn: 0.8718599	total: 3m 2s	remaining: 47.8s
793:	learn: 0.8706293	

931:	learn: 0.7827726	total: 3m 35s	remaining: 15.7s
932:	learn: 0.7825542	total: 3m 35s	remaining: 15.5s
933:	learn: 0.7819097	total: 3m 36s	remaining: 15.3s
934:	learn: 0.7812226	total: 3m 36s	remaining: 15s
935:	learn: 0.7809858	total: 3m 36s	remaining: 14.8s
936:	learn: 0.7806833	total: 3m 36s	remaining: 14.6s
937:	learn: 0.7799977	total: 3m 36s	remaining: 14.3s
938:	learn: 0.7793861	total: 3m 37s	remaining: 14.1s
939:	learn: 0.7789477	total: 3m 37s	remaining: 13.9s
940:	learn: 0.7784523	total: 3m 37s	remaining: 13.6s
941:	learn: 0.7780534	total: 3m 37s	remaining: 13.4s
942:	learn: 0.7773234	total: 3m 38s	remaining: 13.2s
943:	learn: 0.7771623	total: 3m 38s	remaining: 13s
944:	learn: 0.7759755	total: 3m 38s	remaining: 12.7s
945:	learn: 0.7757326	total: 3m 38s	remaining: 12.5s
946:	learn: 0.7750600	total: 3m 39s	remaining: 12.3s
947:	learn: 0.7749859	total: 3m 39s	remaining: 12s
948:	learn: 0.7742992	total: 3m 39s	remaining: 11.8s
949:	learn: 0.7736385	total: 3m 39s	remaining: 11.6s

In [16]:
model = CatBoostRegressor(random_state=42,**params)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(r2_score(y_test,y_pred))

0:	learn: 3.1414201	total: 685ms	remaining: 11m 24s
1:	learn: 3.0666944	total: 1.33s	remaining: 11m 3s
2:	learn: 2.9946841	total: 1.93s	remaining: 10m 40s
3:	learn: 2.9338936	total: 2.5s	remaining: 10m 23s
4:	learn: 2.8725227	total: 3.13s	remaining: 10m 22s
5:	learn: 2.8147283	total: 3.81s	remaining: 10m 31s
6:	learn: 2.7555224	total: 4.44s	remaining: 10m 29s
7:	learn: 2.7028601	total: 5.08s	remaining: 10m 30s
8:	learn: 2.6471750	total: 5.76s	remaining: 10m 33s
9:	learn: 2.5959870	total: 6.47s	remaining: 10m 40s
10:	learn: 2.5451947	total: 7.12s	remaining: 10m 40s
11:	learn: 2.4962209	total: 7.85s	remaining: 10m 46s
12:	learn: 2.4486524	total: 8.55s	remaining: 10m 49s
13:	learn: 2.4012859	total: 9.22s	remaining: 10m 49s
14:	learn: 2.3641994	total: 9.91s	remaining: 10m 50s
15:	learn: 2.3247405	total: 10.6s	remaining: 10m 53s
16:	learn: 2.2881736	total: 11.4s	remaining: 10m 56s
17:	learn: 2.2519763	total: 12.1s	remaining: 10m 58s
18:	learn: 2.2187266	total: 12.8s	remaining: 10m 58s
19:	l

154:	learn: 1.1619690	total: 1m 57s	remaining: 10m 41s
155:	learn: 1.1582810	total: 1m 58s	remaining: 10m 41s
156:	learn: 1.1541066	total: 1m 59s	remaining: 10m 40s
157:	learn: 1.1514369	total: 2m	remaining: 10m 40s
158:	learn: 1.1454448	total: 2m 1s	remaining: 10m 40s
159:	learn: 1.1408624	total: 2m 1s	remaining: 10m 40s
160:	learn: 1.1373963	total: 2m 2s	remaining: 10m 39s
161:	learn: 1.1360838	total: 2m 3s	remaining: 10m 38s
162:	learn: 1.1317148	total: 2m 4s	remaining: 10m 38s
163:	learn: 1.1292301	total: 2m 5s	remaining: 10m 38s
164:	learn: 1.1258246	total: 2m 6s	remaining: 10m 38s
165:	learn: 1.1207499	total: 2m 7s	remaining: 10m 38s
166:	learn: 1.1149917	total: 2m 7s	remaining: 10m 37s
167:	learn: 1.1140497	total: 2m 8s	remaining: 10m 37s
168:	learn: 1.1094704	total: 2m 9s	remaining: 10m 37s
169:	learn: 1.1074154	total: 2m 10s	remaining: 10m 36s
170:	learn: 1.1052401	total: 2m 11s	remaining: 10m 36s
171:	learn: 1.1007236	total: 2m 12s	remaining: 10m 36s
172:	learn: 1.0983579	tot

306:	learn: 0.7574333	total: 3m 56s	remaining: 8m 53s
307:	learn: 0.7551462	total: 3m 57s	remaining: 8m 53s
308:	learn: 0.7537517	total: 3m 58s	remaining: 8m 52s
309:	learn: 0.7523283	total: 3m 58s	remaining: 8m 51s
310:	learn: 0.7497851	total: 3m 59s	remaining: 8m 51s
311:	learn: 0.7476875	total: 4m	remaining: 8m 50s
312:	learn: 0.7464323	total: 4m 1s	remaining: 8m 49s
313:	learn: 0.7461809	total: 4m 1s	remaining: 8m 48s
314:	learn: 0.7450138	total: 4m 2s	remaining: 8m 47s
315:	learn: 0.7440479	total: 4m 3s	remaining: 8m 47s
316:	learn: 0.7427901	total: 4m 4s	remaining: 8m 46s
317:	learn: 0.7406358	total: 4m 5s	remaining: 8m 45s
318:	learn: 0.7394438	total: 4m 5s	remaining: 8m 44s
319:	learn: 0.7365716	total: 4m 6s	remaining: 8m 44s
320:	learn: 0.7342932	total: 4m 7s	remaining: 8m 43s
321:	learn: 0.7336957	total: 4m 8s	remaining: 8m 42s
322:	learn: 0.7310736	total: 4m 8s	remaining: 8m 41s
323:	learn: 0.7292118	total: 4m 9s	remaining: 8m 40s
324:	learn: 0.7282092	total: 4m 10s	remainin

459:	learn: 0.5507137	total: 5m 57s	remaining: 6m 59s
460:	learn: 0.5504985	total: 5m 58s	remaining: 6m 59s
461:	learn: 0.5496086	total: 5m 59s	remaining: 6m 58s
462:	learn: 0.5486517	total: 6m	remaining: 6m 57s
463:	learn: 0.5477464	total: 6m	remaining: 6m 56s
464:	learn: 0.5466957	total: 6m 1s	remaining: 6m 56s
465:	learn: 0.5447013	total: 6m 2s	remaining: 6m 55s
466:	learn: 0.5437928	total: 6m 3s	remaining: 6m 54s
467:	learn: 0.5429297	total: 6m 4s	remaining: 6m 53s
468:	learn: 0.5418255	total: 6m 4s	remaining: 6m 53s
469:	learn: 0.5403054	total: 6m 5s	remaining: 6m 52s
470:	learn: 0.5388871	total: 6m 6s	remaining: 6m 51s
471:	learn: 0.5376496	total: 6m 7s	remaining: 6m 50s
472:	learn: 0.5363210	total: 6m 8s	remaining: 6m 50s
473:	learn: 0.5359772	total: 6m 8s	remaining: 6m 49s
474:	learn: 0.5344949	total: 6m 9s	remaining: 6m 48s
475:	learn: 0.5340725	total: 6m 10s	remaining: 6m 47s
476:	learn: 0.5338217	total: 6m 11s	remaining: 6m 46s
477:	learn: 0.5333523	total: 6m 11s	remaining: 

612:	learn: 0.4211685	total: 8m 15s	remaining: 5m 13s
613:	learn: 0.4201953	total: 8m 17s	remaining: 5m 12s
614:	learn: 0.4197727	total: 8m 18s	remaining: 5m 12s
615:	learn: 0.4190203	total: 8m 20s	remaining: 5m 11s
616:	learn: 0.4179215	total: 8m 21s	remaining: 5m 11s
617:	learn: 0.4173274	total: 8m 23s	remaining: 5m 11s
618:	learn: 0.4170293	total: 8m 25s	remaining: 5m 10s
619:	learn: 0.4161958	total: 8m 26s	remaining: 5m 10s
620:	learn: 0.4154180	total: 8m 28s	remaining: 5m 10s
621:	learn: 0.4149619	total: 8m 30s	remaining: 5m 10s
622:	learn: 0.4142826	total: 8m 32s	remaining: 5m 9s
623:	learn: 0.4138406	total: 8m 33s	remaining: 5m 9s
624:	learn: 0.4133764	total: 8m 35s	remaining: 5m 9s
625:	learn: 0.4129873	total: 8m 36s	remaining: 5m 8s
626:	learn: 0.4123279	total: 8m 38s	remaining: 5m 8s
627:	learn: 0.4116764	total: 8m 40s	remaining: 5m 8s
628:	learn: 0.4114454	total: 8m 41s	remaining: 5m 7s
629:	learn: 0.4105792	total: 8m 43s	remaining: 5m 7s
630:	learn: 0.4101082	total: 8m 45s	

764:	learn: 0.3293576	total: 11m 27s	remaining: 3m 31s
765:	learn: 0.3290491	total: 11m 28s	remaining: 3m 30s
766:	learn: 0.3286559	total: 11m 29s	remaining: 3m 29s
767:	learn: 0.3282513	total: 11m 30s	remaining: 3m 28s
768:	learn: 0.3279412	total: 11m 30s	remaining: 3m 27s
769:	learn: 0.3272868	total: 11m 31s	remaining: 3m 26s
770:	learn: 0.3264611	total: 11m 32s	remaining: 3m 25s
771:	learn: 0.3260095	total: 11m 32s	remaining: 3m 24s
772:	learn: 0.3253518	total: 11m 33s	remaining: 3m 23s
773:	learn: 0.3250622	total: 11m 34s	remaining: 3m 22s
774:	learn: 0.3243056	total: 11m 35s	remaining: 3m 21s
775:	learn: 0.3241956	total: 11m 36s	remaining: 3m 20s
776:	learn: 0.3236473	total: 11m 37s	remaining: 3m 20s
777:	learn: 0.3227902	total: 11m 37s	remaining: 3m 19s
778:	learn: 0.3218804	total: 11m 38s	remaining: 3m 18s
779:	learn: 0.3211837	total: 11m 39s	remaining: 3m 17s
780:	learn: 0.3207004	total: 11m 40s	remaining: 3m 16s
781:	learn: 0.3201704	total: 11m 40s	remaining: 3m 15s
782:	learn

915:	learn: 0.2630054	total: 13m 25s	remaining: 1m 13s
916:	learn: 0.2625692	total: 13m 26s	remaining: 1m 12s
917:	learn: 0.2624060	total: 13m 26s	remaining: 1m 12s
918:	learn: 0.2619565	total: 13m 27s	remaining: 1m 11s
919:	learn: 0.2617812	total: 13m 28s	remaining: 1m 10s
920:	learn: 0.2609081	total: 13m 29s	remaining: 1m 9s
921:	learn: 0.2608139	total: 13m 30s	remaining: 1m 8s
922:	learn: 0.2601476	total: 13m 30s	remaining: 1m 7s
923:	learn: 0.2597812	total: 13m 31s	remaining: 1m 6s
924:	learn: 0.2593801	total: 13m 32s	remaining: 1m 5s
925:	learn: 0.2587021	total: 13m 32s	remaining: 1m 4s
926:	learn: 0.2583932	total: 13m 33s	remaining: 1m 4s
927:	learn: 0.2582943	total: 13m 34s	remaining: 1m 3s
928:	learn: 0.2581488	total: 13m 35s	remaining: 1m 2s
929:	learn: 0.2579096	total: 13m 35s	remaining: 1m 1s
930:	learn: 0.2575618	total: 13m 36s	remaining: 1m
931:	learn: 0.2570536	total: 13m 37s	remaining: 59.6s
932:	learn: 0.2565988	total: 13m 38s	remaining: 58.8s
933:	learn: 0.2564719	tota

# Topology

In [27]:
import pandas as pd
import numpy as np
from pycaret.regression import *
from scipy.stats import gaussian_kde
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from utils import *
from descriptor import *
import warnings

warnings.filterwarnings("ignore")

train_file = "../data/features/train_data.csv"
test_file = "../data/features/test_data.csv"

train_data = pd.read_csv(train_file,index_col=0)
test_data = pd.read_csv(test_file,index_col=0)

data = pd.concat([train_data,test_data],axis=0)

from sklearn.preprocessing import OneHotEncoder
topology = np.array(data["topology"])
topology = topology.reshape(-1,1)
topology_encoder = OneHotEncoder()
topology_en_data = topology_encoder.fit_transform(topology).toarray()

In [28]:
train_topology = topology_en_data[:9261]
test_topology = topology_en_data[9261:]

In [29]:
train_geometric = np.array(train_data[geometric])
test_geometric =np.array(test_data[geometric])
X_train = np.concatenate([train_geometric,train_topology],axis=1)
X_test = np.concatenate([test_geometric,test_topology],axis=1)
y_train = np.array(train_data["GVRH"])
y_test = np.array(test_data["GVRH"])

In [34]:
params = {'learning_rate': 0.051000000000000004, 'depth': 5, 'l2_leaf_reg': 7.0, 'min_child_samples': 32, 'grow_policy': 'Lossguide'}
model = CatBoostRegressor(random_state=42,
                          **params,
                          iterations = 8000) 
model.fit(X_train,y_train)
y_pred_top = model.predict(X_test)
mean_squared_error(y_test,y_pred_top)

0:	learn: 3.1624112	total: 7.72ms	remaining: 1m 1s
1:	learn: 3.1114637	total: 11.6ms	remaining: 46.5s
2:	learn: 3.0618559	total: 15.4ms	remaining: 41.1s
3:	learn: 3.0191669	total: 20.8ms	remaining: 41.5s
4:	learn: 2.9766089	total: 25.5ms	remaining: 40.7s
5:	learn: 2.9347445	total: 29.7ms	remaining: 39.6s
6:	learn: 2.8971734	total: 37.5ms	remaining: 42.9s
7:	learn: 2.8632919	total: 48.5ms	remaining: 48.4s
8:	learn: 2.8296121	total: 66.2ms	remaining: 58.8s
9:	learn: 2.7977660	total: 71.6ms	remaining: 57.2s
10:	learn: 2.7707172	total: 93.2ms	remaining: 1m 7s
11:	learn: 2.7418773	total: 101ms	remaining: 1m 7s
12:	learn: 2.7154772	total: 111ms	remaining: 1m 8s
13:	learn: 2.6924152	total: 116ms	remaining: 1m 6s
14:	learn: 2.6711295	total: 121ms	remaining: 1m 4s
15:	learn: 2.6512087	total: 125ms	remaining: 1m 2s
16:	learn: 2.6313988	total: 130ms	remaining: 1m 1s
17:	learn: 2.6129222	total: 135ms	remaining: 59.9s
18:	learn: 2.5947475	total: 140ms	remaining: 58.7s
19:	learn: 2.5781414	total: 14

172:	learn: 2.1327180	total: 613ms	remaining: 27.7s
173:	learn: 2.1323557	total: 620ms	remaining: 27.9s
174:	learn: 2.1317366	total: 622ms	remaining: 27.8s
175:	learn: 2.1312316	total: 626ms	remaining: 27.8s
176:	learn: 2.1309102	total: 628ms	remaining: 27.7s
177:	learn: 2.1299852	total: 630ms	remaining: 27.7s
178:	learn: 2.1293316	total: 632ms	remaining: 27.6s
179:	learn: 2.1290370	total: 634ms	remaining: 27.5s
180:	learn: 2.1286979	total: 637ms	remaining: 27.5s
181:	learn: 2.1284339	total: 639ms	remaining: 27.4s
182:	learn: 2.1281002	total: 640ms	remaining: 27.4s
183:	learn: 2.1277940	total: 643ms	remaining: 27.3s
184:	learn: 2.1274584	total: 645ms	remaining: 27.3s
185:	learn: 2.1271095	total: 648ms	remaining: 27.2s
186:	learn: 2.1267730	total: 650ms	remaining: 27.2s
187:	learn: 2.1264692	total: 652ms	remaining: 27.1s
188:	learn: 2.1261880	total: 653ms	remaining: 27s
189:	learn: 2.1258819	total: 656ms	remaining: 27s
190:	learn: 2.1256410	total: 658ms	remaining: 26.9s
191:	learn: 2.12

382:	learn: 2.0455406	total: 1.2s	remaining: 23.9s
383:	learn: 2.0454459	total: 1.21s	remaining: 23.9s
384:	learn: 2.0451341	total: 1.21s	remaining: 23.9s
385:	learn: 2.0450315	total: 1.21s	remaining: 23.9s
386:	learn: 2.0446254	total: 1.21s	remaining: 23.9s
387:	learn: 2.0445251	total: 1.22s	remaining: 23.9s
388:	learn: 2.0444119	total: 1.22s	remaining: 23.8s
389:	learn: 2.0442781	total: 1.22s	remaining: 23.8s
390:	learn: 2.0441154	total: 1.22s	remaining: 23.8s
391:	learn: 2.0432974	total: 1.23s	remaining: 23.8s
392:	learn: 2.0432163	total: 1.23s	remaining: 23.8s
393:	learn: 2.0431010	total: 1.23s	remaining: 23.7s
394:	learn: 2.0425326	total: 1.23s	remaining: 23.7s
395:	learn: 2.0420294	total: 1.23s	remaining: 23.7s
396:	learn: 2.0415493	total: 1.24s	remaining: 23.7s
397:	learn: 2.0413676	total: 1.24s	remaining: 23.7s
398:	learn: 2.0412804	total: 1.24s	remaining: 23.7s
399:	learn: 2.0405443	total: 1.24s	remaining: 23.7s
400:	learn: 2.0404343	total: 1.25s	remaining: 23.6s
401:	learn: 2

568:	learn: 1.9938239	total: 1.8s	remaining: 23.6s
569:	learn: 1.9937639	total: 1.81s	remaining: 23.6s
570:	learn: 1.9936790	total: 1.81s	remaining: 23.6s
571:	learn: 1.9935735	total: 1.81s	remaining: 23.6s
572:	learn: 1.9934758	total: 1.82s	remaining: 23.6s
573:	learn: 1.9933254	total: 1.82s	remaining: 23.5s
574:	learn: 1.9926789	total: 1.82s	remaining: 23.5s
575:	learn: 1.9926032	total: 1.83s	remaining: 23.5s
576:	learn: 1.9925363	total: 1.83s	remaining: 23.5s
577:	learn: 1.9924658	total: 1.83s	remaining: 23.5s
578:	learn: 1.9920458	total: 1.83s	remaining: 23.5s
579:	learn: 1.9919520	total: 1.84s	remaining: 23.5s
580:	learn: 1.9915591	total: 1.84s	remaining: 23.5s
581:	learn: 1.9911193	total: 1.85s	remaining: 23.6s
582:	learn: 1.9909394	total: 1.85s	remaining: 23.5s
583:	learn: 1.9882152	total: 1.85s	remaining: 23.6s
584:	learn: 1.9881212	total: 1.86s	remaining: 23.5s
585:	learn: 1.9879188	total: 1.86s	remaining: 23.5s
586:	learn: 1.9876750	total: 1.86s	remaining: 23.5s
587:	learn: 1

792:	learn: 1.9360185	total: 2.39s	remaining: 21.7s
793:	learn: 1.9357734	total: 2.39s	remaining: 21.7s
794:	learn: 1.9356979	total: 2.39s	remaining: 21.7s
795:	learn: 1.9348173	total: 2.4s	remaining: 21.7s
796:	learn: 1.9347554	total: 2.4s	remaining: 21.7s
797:	learn: 1.9346275	total: 2.4s	remaining: 21.7s
798:	learn: 1.9345719	total: 2.4s	remaining: 21.7s
799:	learn: 1.9342040	total: 2.41s	remaining: 21.7s
800:	learn: 1.9338916	total: 2.41s	remaining: 21.7s
801:	learn: 1.9338371	total: 2.41s	remaining: 21.7s
802:	learn: 1.9332619	total: 2.41s	remaining: 21.6s
803:	learn: 1.9329316	total: 2.42s	remaining: 21.6s
804:	learn: 1.9309064	total: 2.42s	remaining: 21.6s
805:	learn: 1.9298028	total: 2.42s	remaining: 21.6s
806:	learn: 1.9297068	total: 2.43s	remaining: 21.6s
807:	learn: 1.9292249	total: 2.45s	remaining: 21.8s
808:	learn: 1.9291705	total: 2.45s	remaining: 21.8s
809:	learn: 1.9290666	total: 2.45s	remaining: 21.8s
810:	learn: 1.9289189	total: 2.46s	remaining: 21.8s
811:	learn: 1.92

...

7271:	learn: 1.1193144	total: 22.5s	remaining: 2.25s
7272:	learn: 1.1193122	total: 22.5s	remaining: 2.25s
7273:	learn: 1.1192093	total: 22.5s	remaining: 2.25s
7274:	learn: 1.1190827	total: 22.5s	remaining: 2.24s
7275:	learn: 1.1190771	total: 22.5s	remaining: 2.24s
7276:	learn: 1.1190550	total: 22.5s	remaining: 2.24s
7277:	learn: 1.1189623	total: 22.5s	remaining: 2.23s
7278:	learn: 1.1188961	total: 22.5s	remaining: 2.23s
7279:	learn: 1.1186429	total: 22.5s	remaining: 2.23s
7280:	learn: 1.1185689	total: 22.5s	remaining: 2.22s
7281:	learn: 1.1183292	total: 22.5s	remaining: 2.22s
7282:	learn: 1.1181936	total: 22.5s	remaining: 2.22s
7283:	learn: 1.1181304	total: 22.5s	remaining: 2.21s
7284:	learn: 1.1180353	total: 22.5s	remaining: 2.21s
7285:	learn: 1.1178028	total: 22.5s	remaining: 2.21s
7286:	learn: 1.1177990	total: 22.5s	remaining: 2.21s
7287:	learn: 1.1176246	total: 22.6s	remaining: 2.2s
7288:	learn: 1.1175334	total: 22.6s	remaining: 2.2s
7289:	learn: 1.1175321	total: 22.6s	remaining: 2

7480:	learn: 1.1052343	total: 23.1s	remaining: 1.6s
7481:	learn: 1.1050029	total: 23.1s	remaining: 1.6s
7482:	learn: 1.1049238	total: 23.1s	remaining: 1.59s
7483:	learn: 1.1047956	total: 23.1s	remaining: 1.59s
7484:	learn: 1.1047598	total: 23.1s	remaining: 1.59s
7485:	learn: 1.1047305	total: 23.1s	remaining: 1.59s
7486:	learn: 1.1046451	total: 23.1s	remaining: 1.58s
7487:	learn: 1.1044718	total: 23.1s	remaining: 1.58s
7488:	learn: 1.1041217	total: 23.1s	remaining: 1.58s
7489:	learn: 1.1040390	total: 23.1s	remaining: 1.57s
7490:	learn: 1.1040357	total: 23.1s	remaining: 1.57s
7491:	learn: 1.1039590	total: 23.1s	remaining: 1.57s
7492:	learn: 1.1038398	total: 23.1s	remaining: 1.56s
7493:	learn: 1.1038358	total: 23.1s	remaining: 1.56s
7494:	learn: 1.1038234	total: 23.1s	remaining: 1.56s
7495:	learn: 1.1037552	total: 23.1s	remaining: 1.55s
7496:	learn: 1.1036008	total: 23.1s	remaining: 1.55s
7497:	learn: 1.1035658	total: 23.1s	remaining: 1.55s
7498:	learn: 1.1035641	total: 23.1s	remaining: 1

7691:	learn: 1.0894178	total: 23.7s	remaining: 948ms
7692:	learn: 1.0893959	total: 23.7s	remaining: 945ms
7693:	learn: 1.0893219	total: 23.7s	remaining: 942ms
7694:	learn: 1.0892135	total: 23.7s	remaining: 939ms
7695:	learn: 1.0892124	total: 23.7s	remaining: 936ms
7696:	learn: 1.0891929	total: 23.7s	remaining: 933ms
7697:	learn: 1.0891829	total: 23.7s	remaining: 930ms
7698:	learn: 1.0890417	total: 23.7s	remaining: 927ms
7699:	learn: 1.0889563	total: 23.7s	remaining: 923ms
7700:	learn: 1.0889370	total: 23.7s	remaining: 920ms
7701:	learn: 1.0887879	total: 23.7s	remaining: 917ms
7702:	learn: 1.0884727	total: 23.7s	remaining: 914ms
7703:	learn: 1.0884132	total: 23.7s	remaining: 911ms
7704:	learn: 1.0884065	total: 23.7s	remaining: 908ms
7705:	learn: 1.0883384	total: 23.7s	remaining: 905ms
7706:	learn: 1.0882659	total: 23.7s	remaining: 902ms
7707:	learn: 1.0881479	total: 23.7s	remaining: 899ms
7708:	learn: 1.0881376	total: 23.7s	remaining: 896ms
7709:	learn: 1.0881194	total: 23.7s	remaining:

7900:	learn: 1.0743429	total: 24.3s	remaining: 304ms
7901:	learn: 1.0743137	total: 24.3s	remaining: 301ms
7902:	learn: 1.0743121	total: 24.3s	remaining: 298ms
7903:	learn: 1.0743108	total: 24.3s	remaining: 295ms
7904:	learn: 1.0742654	total: 24.3s	remaining: 292ms
7905:	learn: 1.0740791	total: 24.3s	remaining: 289ms
7906:	learn: 1.0740769	total: 24.3s	remaining: 286ms
7907:	learn: 1.0740328	total: 24.3s	remaining: 283ms
7908:	learn: 1.0739956	total: 24.3s	remaining: 279ms
7909:	learn: 1.0738886	total: 24.3s	remaining: 276ms
7910:	learn: 1.0738722	total: 24.3s	remaining: 273ms
7911:	learn: 1.0737940	total: 24.3s	remaining: 270ms
7912:	learn: 1.0737228	total: 24.3s	remaining: 267ms
7913:	learn: 1.0736870	total: 24.3s	remaining: 264ms
7914:	learn: 1.0736859	total: 24.3s	remaining: 261ms
7915:	learn: 1.0736851	total: 24.3s	remaining: 258ms
7916:	learn: 1.0736511	total: 24.3s	remaining: 255ms
7917:	learn: 1.0736126	total: 24.3s	remaining: 252ms
7918:	learn: 1.0736116	total: 24.3s	remaining:

4.046432414496558

In [ ]:
import optuna
from sklearn import datasets

def objective(trial,X_train = X_train, y_train = y_train):
    
        
    learning_rate = trial.suggest_discrete_uniform("learning_rate", 0.001, 0.1, 0.005)
  
    depth = trial.suggest_int('depth', 5, 10)
                
    l2_leaf_reg = trial.suggest_discrete_uniform('l2_leaf_reg', 2.0, 30.0, 0.5)
        
    min_child_samples = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
        
    grow_policy = trial.suggest_categorical("grow_policy",["SymmetricTree", "Depthwise", "Lossguide"])
    
    accuracy = []
    
    kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

    loss = []

    for train_index2, valid_index2 in kf.split(X_train):

        X_train2, X_valid2 = X_train[train_index2], X_train[valid_index2]
        y_train2, y_valid2 = y_train[train_index2], y_train[valid_index2]

        model = CatBoostRegressor(random_state=42,
                                  learning_rate=learning_rate,
                                  depth = depth,
                                  l2_leaf_reg = l2_leaf_reg,
                                  min_child_samples = min_child_samples,
                                  grow_policy = grow_policy,
                                  silent = True) 

        model.fit(X_train2, y_train2) 

        y_pred2 = model.predict(X_valid2) 
        
        score = r2_score(y_valid2,y_pred2)
        
        rmse = mean_squared_error(y_valid2,y_pred2,squared = False)
        
        loss.append(rmse)
        
        accuracy.append(score)
        
    average_loss = np.mean(loss)
    average_score = np.mean(accuracy)    
    y_pred = model.predict(X_test)
    
    print(r2_score(y_test,y_pred))
    
    return average_score
    
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials = 1000)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print("best r2 is :",study.best_trial.values)

In [41]:
print(r2_score(y_test,y_pred_top))

0.7381071773264455